In [1]:
from umap import UMAP
import pandas as pd
import numpy as np
import openml
from sklearn.linear_model import LinearRegression 
import lightgbm as lgbm
import optuna
import gower
from sklearn.gaussian_process import GaussianProcessRegressor
import gpytorch
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process.kernels import Matern
from engression import engression, engression_bagged
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from rtdl_revisiting_models import MLP, ResNet, FTTransformer
from sklearn.metrics.pairwise import euclidean_distances

c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#openml.config.apikey = 'FILL_IN_OPENML_API_KEY'  # set the OpenML Api Key
SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite

task = openml.tasks.get_task(361072)  # download the OpenML task
dataset = task.get_dataset()

X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)

c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [3]:
# Apply UMAP decomposition
umap = UMAP(n_components=2, random_state=42)
X_umap = umap.fit_transform(X)

# calculate the Euclidean distance matrix
euclidean_dist_matrix = euclidean_distances(X_umap)

# calculate the Euclidean distance for each data point
euclidean_dist = np.mean(euclidean_dist_matrix, axis=1)

euclidean_dist = pd.Series(euclidean_dist, index=X.index)
far_index = euclidean_dist.index[np.where(euclidean_dist >= np.quantile(euclidean_dist, 0.8))[0]]
close_index = euclidean_dist.index[np.where(euclidean_dist < np.quantile(euclidean_dist, 0.8))[0]]

X_train = X.loc[close_index,:]
X_test = X.loc[far_index,:]
y_train = y.loc[close_index]
y_test = y.loc[far_index]

# Apply UMAP decomposition on the training set
X_umap_train = umap.fit_transform(X_train)

# calculate the Euclidean distance matrix for the training set
euclidean_dist_matrix_train = euclidean_distances(X_umap_train)

# calculate the Euclidean distance for each data point in the training set
euclidean_dist_train = np.mean(euclidean_dist_matrix_train, axis=1)

euclidean_dist_train = pd.Series(euclidean_dist_train, index=X_train.index)
far_index_train = euclidean_dist_train.index[np.where(euclidean_dist_train >= np.quantile(euclidean_dist_train, 0.8))[0]]
close_index_train = euclidean_dist_train.index[np.where(euclidean_dist_train < np.quantile(euclidean_dist_train, 0.8))[0]]

X_train_ = X_train.loc[close_index_train,:]
X_val = X_train.loc[far_index_train,:]
y_train_ = y_train.loc[close_index_train]
y_val = y_train.loc[far_index_train]

c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [6]:
X_train_tensor = torch.tensor(X_train_.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_.values, dtype=torch.float32)yy

In [9]:
X_train_tensor.shape
y_train_tensor.shape

torch.Size([5242])

In [30]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
import gpytorch
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

lengthscale=1

class SVGPMODEL(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(SVGPMODEL, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=inducing_points.shape[1], lengthscale=lengthscale),
            ard_num_dims=inducing_points.shape[1]
        )
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    

    # Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_.values, dtype=torch.float32)

    # Initialize the Gaussian Process model and likelihood
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = SVGPMODEL(X_train_tensor)

    # Set the model in training mode
model.train()
likelihood.train()

    # Define the learning params
n_epochs=5 #trial.suggest_int('n_epochs', 100, 5000)
learning_rate=0.001

    # Use the negative log likelihood as the loss
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=y_train_tensor.numel())
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()
    output = model(X_train_tensor)
    print(output)
    loss = -mll(output, y_train_tensor)
    print(loss)
    loss.backward()
    print(model.covar_module.base_kernel.lengthscale)
    optimizer.step()

    # Set the model in evaluation mode
model.eval()
likelihood.eval()

    # Make predictions on the validation set
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    y_pred = model(torch.tensor(X_val.values, dtype=torch.float32))

    # Calculate RMSE
RMSE_SVGP = torch.sqrt(torch.mean(torch.square(torch.tensor(y_val.values) - y_pred.mean)))

MultivariateNormal(loc: torch.Size([5242]))
tensor(6009.2500, grad_fn=<NegBackward0>)
tensor([[0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931]], grad_fn=<SoftplusBackward0>)
MultivariateNormal(loc: torch.Size([5242]))
tensor(6009.0293, grad_fn=<NegBackward0>)
tensor([[0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931]], grad_fn=<SoftplusBackward0>)
MultivariateNormal(loc: torch.Size([5242]))
tensor(6008.8096, grad_fn=<NegBackward0>)
tensor([[0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931]], grad_fn=<SoftplusBackward0>)
MultivariateNormal(loc: torch.Size([5242]))
tensor(6008.5894, gr

In [25]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
import gpytorch
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

lengthscale=1

'''class SVGPMODEL(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(SVGPMODEL, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=inducing_points.shape[1], lengthscale=lengthscale),
            ard_num_dims=inducing_points.shape[1]
        )
        
        def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)'''
class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        # Define a prior for the lengthscale
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1], lengthscale=lengthscale),
            ard_num_dims=train_x.shape[1]
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


    # Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_.values, dtype=torch.float32)

    # Initialize the Gaussian Process model and likelihood
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPModel(X_train_tensor, y_train_tensor, likelihood) #(X_train_tensor)

    # Set the model in training mode
model.train()
likelihood.train()

    # Define the learning params
n_epochs=5 #trial.suggest_int('n_epochs', 100, 5000)
learning_rate=0.001

    # Use the negative log likelihood as the loss
#mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=y_train_tensor.numel())
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()
    output = model(X_train_tensor)
    print(output)
    loss = -mll(output, y_train_tensor)
    print(loss)
    loss.backward()
    print(model.covar_module.base_kernel.lengthscale)
    optimizer.step()

    # Set the model in evaluation mode
model.eval()
likelihood.eval()

    # Make predictions on the validation set
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    y_pred = model(torch.tensor(X_val.values, dtype=torch.float32))

    # Calculate RMSE
RMSE_SVGP = torch.sqrt(torch.mean(torch.square(torch.tensor(y_val.values) - y_pred.mean)))

MultivariateNormal(loc: torch.Size([5242]))
tensor(3550.2903, grad_fn=<NegBackward0>)
tensor([[0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931]], grad_fn=<SoftplusBackward0>)
MultivariateNormal(loc: torch.Size([5242]))
tensor(3547.6541, grad_fn=<NegBackward0>)
tensor([[0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931]], grad_fn=<SoftplusBackward0>)
MultivariateNormal(loc: torch.Size([5242]))
tensor(3545.0198, grad_fn=<NegBackward0>)
tensor([[0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931]], grad_fn=<SoftplusBackward0>)
MultivariateNormal(loc: torch.Size([5242]))
tensor(3542.3904, gr

In [29]:
# model.covar_module.base_kernel.lengthscale
model

GPModel(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): MaternKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
)

#### Gaussian process with stochastic variational inference

In [12]:
import gpytorch
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

N_TRIALS=2

def SVGP_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    lengthscale=trial.suggest_float('lengthscale', 1e-8, 10, log=True)

    class SVGPMODEL(gpytorch.models.ApproximateGP):
        def __init__(self, inducing_points):
            variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
            variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
            super(SVGPMODEL, self).__init__(variational_strategy)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=inducing_points.shape[1], lengthscale=lengthscale),
                ard_num_dims=inducing_points.shape[1]
            )

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_.values, dtype=torch.float32)

    # Initialize the Gaussian Process model and likelihood
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = SVGPMODEL(X_train_tensor)

    # Set the model in training mode
    model.train()
    likelihood.train()

    # Define the learning params
    n_epochs=5 #trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)

    # Use the negative log likelihood as the loss
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=y_train_tensor.numel())
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        output = model(X_train_tensor)
        print(output)
        loss = -mll(output, y_train_tensor)
        print(loss)
        loss.backward()
        optimizer.step()

    # Set the model in evaluation mode
    model.eval()
    likelihood.eval()

    # Make predictions on the validation set
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        y_pred = model(torch.tensor(X_val.values, dtype=torch.float32))

    # Calculate RMSE
    RMSE_SVGP = torch.sqrt(torch.mean(torch.square(torch.tensor(y_val.values) - y_pred.mean)))

    return RMSE_SVGP

sampler_SVGP = optuna.samplers.TPESampler(seed=10)
study_SVGP = optuna.create_study(sampler=sampler_SVGP, direction='minimize')
study_SVGP.optimize(SVGP_opt, n_trials=N_TRIALS)

[I 2024-01-12 10:45:07,387] A new study created in memory with name: no-name-2b329cdc-aab8-4e91-9a9e-c2d140ff1f4e
[I 2024-01-12 10:49:35,844] Trial 0 finished with value: 79.97399139404297 and parameters: {'lengthscale': 0.08747537025773001, 'learning_rate': 0.00011376505702653915}. Best is trial 0 with value: 79.97399139404297.
[W 2024-01-12 10:50:41,288] Trial 1 failed with parameters: {'lengthscale': 0.005044685709888605, 'learning_rate': 0.010495405390719734} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\dalma\AppData\Local\Temp\ipykernel_24668\2789280030.py", line 55, in SVGP_opt
    output = model(X_train_tensor)
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\gpytorch\models\approximate_gp.py", line 108, in __call__
    retu

KeyboardInterrupt: 

In [ ]:
# Access the best parameters
best_params_SVGP = study_SVGP.best_params
lengthscale = best_params_SVGP['lengthscale']
n_epochs = best_params_SVGP['n_epochs']
learning_rate = best_params_SVGP['learning_rate']

class SVGPMODEL(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(SVGPMODEL, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=inducing_points.shape[1], lengthscale=lengthscale),
            ard_num_dims=inducing_points.shape[1]
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.float32)

# Initialize the final Gaussian Process model with the best parameters
likelihood = gpytorch.likelihoods.GaussianLikelihood()
final_model = SVGPMODEL(X_tensor)

# Set the model in training mode
final_model.train()
likelihood.train()

# Use the negative log likelihood as the loss
mll = gpytorch.mlls.VariationalELBO(likelihood, final_model, num_data=y_tensor.numel())
optimizer = torch.optim.Adam(final_model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()
    output = final_model(X_tensor)
    loss = -mll(output, y_tensor)
    loss.backward()
    optimizer.step()

# Set the final model in evaluation mode
final_model.eval()
likelihood.eval()

# Make predictions on the validation set
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    y_pred = final_model(torch.tensor(X_test.values, dtype=torch.float32))

# Calculate RMSE
RMSE_SVGP = torch.sqrt(torch.mean(torch.square(torch.tensor(y_test.values) - y_pred.mean)))
print("RMSE SVGP: ", RMSE_SVGP)

KeyError: 'lengthscale'

#### Gaussian process

In [ ]:
N_TRIALS=5

def GP_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    lengthscale=trial.suggest_float('lengthscale', 1e-8, 10, log=True)

    class GPModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            # Define a prior for the lengthscale
            self.covar_module = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1], lengthscale=lengthscale),
                ard_num_dims=train_x.shape[1]
            )

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_.values, dtype=torch.float32)

    # Initialize the Gaussian Process model and likelihood
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(X_train_tensor, y_train_tensor, likelihood)

    # Set the model in training mode
    model.train()
    likelihood.train()

    # Define the learning params
    n_epochs=1#trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)

    # Use the negative log likelihood as the loss
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        output = model(X_train_tensor)
        loss = -mll(output, y_train_tensor)
        loss.backward()
        optimizer.step()

    # Set the model in evaluation mode
    model.eval()
    likelihood.eval()

    # Make predictions on the validation set
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        y_pred = model(torch.tensor(X_val.values, dtype=torch.float32))

    # Calculate RMSE
    RMSE_GP = torch.sqrt(torch.mean(torch.square(torch.tensor(y_val.values) - y_pred.mean)))

    return RMSE_GP

sampler_GP = optuna.samplers.TPESampler(seed=10)
study_GP = optuna.create_study(sampler=sampler_GP, direction='minimize')
study_GP.optimize(GP_opt, n_trials=N_TRIALS)

[I 2024-01-09 15:39:52,596] A new study created in memory with name: no-name-5b64c314-cee9-4253-bb1a-6badf9a912e4
[W 2024-01-09 16:24:17,381] Trial 0 failed with parameters: {'loc': 0.7724640400504122, 'scale': 0.011355222730341348, 'n_epochs': 3205, 'learning_rate': 0.03746531373867673} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\dalma\AppData\Local\Temp\ipykernel_29424\1361086071.py", line 54, in GP_opt
    loss.backward()
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_tensor.py", line 492, in backward
    torch.autograd.backward(
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\autograd\__init__.py", line 251, in backward
    Variable._execution_engine.run_backward(  # Calls into t

KeyboardInterrupt: 

In [ ]:
# Access the best parameters
best_params_GP = study_GP.best_params
lengthscale = best_params_SVGP['lengthscale']
n_epochs=best_params_GP['n_epochs']
learning_rate=best_params_GP['learning_rate']

class GPModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            # Define a prior for the lengthscale
            self.covar_module = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1], lengthscale=lengthscale),
                ard_num_dims=train_x.shape[1]
            )

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.float32)

# Initialize the final Gaussian Process model with the best parameters
likelihood = gpytorch.likelihoods.GaussianLikelihood()
final_model = GPModel(X_tensor, y_tensor, likelihood)

# Set the model in training mode
final_model.train()
likelihood.train()

# Use the negative log likelihood as the loss
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, final_model)
optimizer = torch.optim.Adam(final_model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()
    output = final_model(X_tensor)
    loss = -mll(output, y_tensor)
    loss.backward()
    optimizer.step()

# Set the final model in evaluation mode
final_model.eval()
likelihood.eval()

# Make predictions on the validation set
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    y_pred = final_model(torch.tensor(X_test.values, dtype=torch.float32))

# Calculate RMSE
RMSE_GP = torch.sqrt(torch.mean(torch.square(torch.tensor(y_test.values) - y_pred.mean)))
print("RMSE GP: ", RMSE_GP)

#### MLP

In [ ]:
N_TRIALS=5

d_out = 1  
d_in=X_train_.shape[1]

def MLP_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    n_blocks = trial.suggest_int("n_blocks", 1, 5)
    d_block = trial.suggest_int("d_block", 10, 500)
    dropout = trial.suggest_float("dropout", 0, 1)

    MLP_model = MLP(
    d_in=d_in,
    d_out=d_out,
    n_blocks=n_blocks,
    d_block=d_block,
    dropout=dropout,
    )
    n_epochs=trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)
    optimizer=torch.optim.Adam(MLP_model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()
    loss_Adam=[]

    for i in range(n_epochs):
        # making a pridiction in forward pass
        y_train_hat = MLP_model(torch.Tensor(X_train_.values)).reshape(-1,)
        # calculating the loss between original and predicted data points
        loss = criterion(y_train_hat, torch.Tensor(y_train_.values))
        # store loss into list
        loss_Adam.append(loss.item())
        # zeroing gradients after each iteration
        optimizer.zero_grad()
        # backward pass for computing the gradients of the loss w.r.t to learnable parameters
        loss.backward()
        # updating the parameters after each iteration
        optimizer.step()

    y_val_hat_MLP=(MLP_model(torch.Tensor(X_val.values)).reshape(-1,)).detach().numpy()
    RMSE_MLP=np.sqrt(np.mean((y_val-y_val_hat_MLP)**2))

    return RMSE_MLP

sampler_MLP = optuna.samplers.TPESampler(seed=10)
study_MLP = optuna.create_study(sampler=sampler_MLP, direction='minimize')
study_MLP.optimize(MLP_opt, n_trials=N_TRIALS)

[I 2023-12-23 17:24:18,248] A new study created in memory with name: no-name-da428c83-3742-4845-8cb0-a33c232b7db0
[I 2023-12-23 17:24:38,519] Trial 0 finished with value: 60.821720123291016 and parameters: {'n_blocks': 4, 'd_block': 20, 'dropout': 0.6336482349262754, 'n_epochs': 3769, 'learning_rate': 0.02497549991389926}. Best is trial 0 with value: 60.821720123291016.
[I 2023-12-23 17:24:48,376] Trial 1 finished with value: 502.2777099609375 and parameters: {'n_blocks': 2, 'd_block': 107, 'dropout': 0.7605307121989587, 'n_epochs': 928, 'learning_rate': 0.004508156727283112}. Best is trial 0 with value: 60.821720123291016.
[I 2023-12-23 17:35:15,252] Trial 2 finished with value: 25.43311882019043 and parameters: {'n_blocks': 4, 'd_block': 478, 'dropout': 0.003948266327914451, 'n_epochs': 2610, 'learning_rate': 0.040649785986440465}. Best is trial 2 with value: 25.43311882019043.
[I 2023-12-23 17:46:23,421] Trial 3 finished with value: 79.76200103759766 and parameters: {'n_blocks': 4, 

In [ ]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

MLP_model = MLP(
    d_in=d_in,
    d_out=d_out,
    n_blocks=study_MLP.best_params['n_blocks'],
    d_block=study_MLP.best_params['d_block'],
    dropout=study_MLP.best_params['dropout'],
    )
n_epochs=study_MLP.best_params['n_epochs']
learning_rate=study_MLP.best_params['learning_rate']
optimizer=torch.optim.Adam(MLP_model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()
loss_Adam=[]

for i in range(n_epochs):
    # making a pridiction in forward pass
    y_train_hat = MLP_model(torch.Tensor(X_train.values)).reshape(-1,)
    # calculating the loss between original and predicted data points
    loss = criterion(y_train_hat, torch.Tensor(y_train.values))
    # store loss into list
    loss_Adam.append(loss.item())
    # zeroing gradients after each iteration
    optimizer.zero_grad()
    # backward pass for computing the gradients of the loss w.r.t to learnable parameters
    loss.backward()
    # updating the parameters after each iteration
    optimizer.step()

y_test_hat_MLP=(MLP_model(torch.Tensor(X_test.values)).reshape(-1,)).detach().numpy()
RMSE_MLP=np.sqrt(np.mean((y_test-y_test_hat_MLP)**2))
print("RMSE MLP: ", RMSE_MLP)

RMSE MLP:  2127.5535


#### ResNet

In [ ]:
N_TRIALS=5

d_out = 1  
d_in=X_train_.shape[1]

def ResNet_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    n_blocks = trial.suggest_int("n_blocks", 1, 5)
    d_block = trial.suggest_int("d_block", 10, 500)
    dropout1 = trial.suggest_float("dropout1", 0, 1)
    dropout2 = trial.suggest_float("dropout2", 0, 1)
    d_hidden_multiplier=trial.suggest_float("d_hidden_multiplier", 0.5, 3)

    ResNet_model = ResNet(
    d_in=d_in,
    d_out=d_out,
    n_blocks=n_blocks,
    d_block=d_block,
    d_hidden=None,
    d_hidden_multiplier=d_hidden_multiplier,
    dropout1=dropout1,
    dropout2=dropout2,
    )
    n_epochs=trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)
    optimizer=torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()
    loss_Adam=[]

    for i in range(n_epochs):
        # making a pridiction in forward pass
        y_train_hat = ResNet_model(torch.Tensor(X_train_.values)).reshape(-1,)
        # calculating the loss between original and predicted data points
        loss = criterion(y_train_hat, torch.Tensor(y_train_.values))
        # store loss into list
        loss_Adam.append(loss.item())
        # zeroing gradients after each iteration
        optimizer.zero_grad()
        # backward pass for computing the gradients of the loss w.r.t to learnable parameters
        loss.backward()
        # updating the parameters after each iteration
        optimizer.step()

    y_val_hat_ResNet=(ResNet_model(torch.Tensor(X_val.values)).reshape(-1,)).detach().numpy()
    RMSE_ResNet=np.sqrt(np.mean((y_val-y_val_hat_ResNet)**2))

    return RMSE_ResNet

sampler_ResNet = optuna.samplers.TPESampler(seed=10)
study_ResNet = optuna.create_study(sampler=sampler_ResNet, direction='minimize')
study_ResNet.optimize(ResNet_opt, n_trials=N_TRIALS)

[I 2023-12-23 18:10:31,640] A new study created in memory with name: no-name-944004cd-5e6d-4aa6-9fe6-432d77df0455


[I 2023-12-23 18:10:52,768] Trial 0 finished with value: 30.78984260559082 and parameters: {'n_blocks': 4, 'd_block': 20, 'dropout1': 0.6336482349262754, 'dropout2': 0.7488038825386119, 'd_hidden_multiplier': 1.7462675307564761, 'n_epochs': 1201, 'learning_rate': 0.009983336951505236}. Best is trial 0 with value: 30.78984260559082.
[I 2023-12-23 18:11:04,384] Trial 1 finished with value: 34.90650177001953 and parameters: {'n_blocks': 4, 'd_block': 93, 'dropout1': 0.08833981417401027, 'dropout2': 0.6853598183677972, 'd_hidden_multiplier': 2.8834833654873413, 'n_epochs': 119, 'learning_rate': 0.02565839394295025}. Best is trial 0 with value: 30.78984260559082.
[I 2023-12-27 08:43:55,096] Trial 2 finished with value: 23.0227108001709 and parameters: {'n_blocks': 5, 'd_block': 310, 'dropout1': 0.7217553174317995, 'dropout2': 0.29187606817063316, 'd_hidden_multiplier': 2.7944353062823586, 'n_epochs': 3602, 'learning_rate': 0.027172963963761936}. Best is trial 2 with value: 23.0227108001709.

In [ ]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

ResNet_model = ResNet(
    d_in=d_in,
    d_out=d_out,
    n_blocks=study_ResNet.best_params['n_blocks'],
    d_block=study_ResNet.best_params['d_block'],
    d_hidden=None,
    d_hidden_multiplier=study_ResNet.best_params['d_hidden_multiplier'],
    dropout1=study_ResNet.best_params['dropout1'],
    dropout2=study_ResNet.best_params['dropout2'],
    )
n_epochs=study_ResNet.best_params['n_epochs']
learning_rate=study_ResNet.best_params['learning_rate']
optimizer=torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()
loss_Adam=[]

for i in range(n_epochs):
    # making a pridiction in forward pass
    y_train_hat = ResNet_model(torch.Tensor(X_train.values)).reshape(-1,)
    # calculating the loss between original and predicted data points
    loss = criterion(y_train_hat, torch.Tensor(y_train.values))
    # store loss into list
    loss_Adam.append(loss.item())
    # zeroing gradients after each iteration
    optimizer.zero_grad()
    # backward pass for computing the gradients of the loss w.r.t to learnable parameters
    loss.backward()
    # updating the parameters after each iteration
    optimizer.step()

y_test_hat_ResNet=(ResNet_model(torch.Tensor(X_test.values)).reshape(-1,)).detach().numpy()
RMSE_ResNet=np.sqrt(np.mean((y_test-y_test_hat_ResNet)**2))
print("RMSE ResNet: ", RMSE_ResNet)

RMSE ResNet:  28.044754


#### FFTransformer

In [ ]:
N_TRIALS=5

d_out = 1  
d_in=X_train_.shape[1]

def FTTrans_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    n_blocks = trial.suggest_int("n_blocks", 1, 5)
    d_block_multiplier = trial.suggest_int("d_block_multiplier", 1, 25)
    attention_n_heads = trial.suggest_int("attention_n_heads", 1, 20)
    attention_dropout = trial.suggest_float("attention_dropout", 0, 1)
    ffn_d_hidden_multiplier=trial.suggest_float("ffn_d_hidden_multiplier", 0.5, 3)
    ffn_dropout = trial.suggest_float("ffn_dropout", 0, 1)
    residual_dropout = trial.suggest_float("residual_dropout", 0, 1)

    FTTrans_model = FTTransformer(
    n_cont_features=d_in,
    cat_cardinalities=[],
    d_out=d_out,
    n_blocks=n_blocks,
    d_block=d_block_multiplier*attention_n_heads,
    attention_n_heads=attention_n_heads,
    attention_dropout=attention_dropout,
    ffn_d_hidden=None,
    ffn_d_hidden_multiplier=ffn_d_hidden_multiplier,
    ffn_dropout=ffn_dropout,
    residual_dropout=residual_dropout,
    )

    n_epochs=trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)
    optimizer=torch.optim.Adam(FTTrans_model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()
    loss_Adam=[]

    for i in range(n_epochs):
        # making a pridiction in forward pass
        y_train_hat = FTTrans_model(torch.Tensor(X_train_.values),None).reshape(-1,)
        # calculating the loss between original and predicted data points
        loss = criterion(y_train_hat, torch.Tensor(y_train_.values))
        # store loss into list
        loss_Adam.append(loss.item())
        # zeroing gradients after each iteration
        optimizer.zero_grad()
        # backward pass for computing the gradients of the loss w.r.t to learnable parameters
        loss.backward()
        # updating the parameters after each iteration
        optimizer.step()

    y_val_hat_FTTrans=(FTTrans_model(torch.Tensor(X_val.values), None).reshape(-1,)).detach().numpy()
    RMSE_FTTrans=np.sqrt(np.mean((y_val-y_val_hat_FTTrans)**2))

    return RMSE_FTTrans

sampler_FTTrans = optuna.samplers.TPESampler(seed=10)
study_FTTrans = optuna.create_study(sampler=sampler_FTTrans, direction='minimize')
study_FTTrans.optimize(FTTrans_opt, n_trials=N_TRIALS)

[I 2024-01-08 14:46:52,555] A new study created in memory with name: no-name-7acaca2b-54b2-449c-8afa-679aaa0aa628


In [ ]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

FTTrans_model = FTTransformer(
    n_cont_features=d_in,
    cat_cardinalities=[],
    d_out=d_out,
    n_blocks=study_FTTrans.best_params['n_blocks'],
    d_block=study_FTTrans.best_params['d_block'],
    attention_n_heads=study_FTTrans.best_params['attention_n_heads'],
    attention_dropout=study_FTTrans.best_params['attention_dropout'],
    ffn_d_hidden=None,
    ffn_d_hidden_multiplier=study_FTTrans.best_params['ffn_d_hidden_multiplier'],
    ffn_dropout=study_FTTrans.best_params['ffn_dropout'],
    residual_dropout=study_FTTrans.best_params['residual_dropout'],
    )
n_epochs=study_FTTrans.best_params['n_epochs']
learning_rate=study_FTTrans.best_params['learning_rate']
optimizer=torch.optim.Adam(FTTrans_model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()
loss_Adam=[]

for i in range(n_epochs):
    # making a pridiction in forward pass
    y_train_hat = FTTrans_model(torch.Tensor(X_train.values), None).reshape(-1,)
    # calculating the loss between original and predicted data points
    loss = criterion(y_train_hat, torch.Tensor(y_train.values))
    # store loss into list
    loss_Adam.append(loss.item())
    # zeroing gradients after each iteration
    optimizer.zero_grad()
    # backward pass for computing the gradients of the loss w.r.t to learnable parameters
    loss.backward()
    # updating the parameters after each iteration
    optimizer.step()

y_test_hat_FTTrans=(FTTrans_model(torch.Tensor(X_test.values)).reshape(-1,)).detach().numpy()
RMSE_FTTrans=np.sqrt(np.mean((y_test-y_test_hat_FTTrans)**2))
print("RMSE FTTrans: ", RMSE_FTTrans)

#### Boosted trees, random forest, engression, linear regression

In [ ]:
N_TRIALS=5

def boosted(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5, log=True),
              'n_estimators': trial.suggest_int('n_estimators', 100, 500),
              'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
              'max_depth': trial.suggest_int('max_depth', 1, 30),
              'min_child_samples': trial.suggest_int('min_child_samples', 10, 100)}
    
    boosted_tree_model=lgbm.LGBMRegressor(**params)
    boosted_tree_model.fit(X_train_, y_train_)
    y_val_hat_boost=boosted_tree_model.predict(X_val)
    RMSE_boost=np.sqrt(np.mean((y_val-y_val_hat_boost)**2))

    return RMSE_boost

sampler_boost = optuna.samplers.TPESampler(seed=10)
study_boost = optuna.create_study(sampler=sampler_boost, direction='minimize')
study_boost.optimize(boosted, n_trials=N_TRIALS)

boosted_model=lgbm.LGBMRegressor(**study_boost.best_params)

def rf(trial):

    params = {'n_estimators': trial.suggest_int('n_estimators', 100, 500),
              'max_depth': trial.suggest_int('max_depth', 1, 30),
              'max_features': trial.suggest_int('max_features', 1, 30),
              'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 100)}
    
    rf_model=RandomForestRegressor(**params)
    rf_model.fit(X_train_, y_train_)
    y_val_hat_rf=rf_model.predict(X_val)
    RMSE_rf=np.sqrt(np.mean((y_val-y_val_hat_rf)**2))

    return RMSE_rf

sampler_rf = optuna.samplers.TPESampler(seed=10)
study_rf = optuna.create_study(sampler=sampler_rf, direction='minimize')
study_rf.optimize(rf, n_trials=N_TRIALS)

rf_model=RandomForestRegressor(**study_rf.best_params)


def engressor_NN(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5, log=True),
              'num_epoches': trial.suggest_int('num_epoches', 100, 1000),
              'num_layer': trial.suggest_int('num_layer', 2, 5),
              'hidden_dim': trial.suggest_int('hidden_dim', 50, 100),
              'noise_dim': trial.suggest_int('noise_dim', 50, 100),}
    
    engressor_model=engression(torch.Tensor(np.array(X_train_)), torch.Tensor(np.array(y_train_).reshape(-1,1)), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=1000)
    y_val_hat_engression=engressor_model.predict(torch.Tensor(np.array(X_val)), target="mean")
    RMSE_engression=np.sqrt((((torch.Tensor(np.array(y_val).reshape(-1,1)))-y_val_hat_engression)**2).mean(axis=0))

    return RMSE_engression

sampler_engression = optuna.samplers.TPESampler(seed=10)
study_engression = optuna.create_study(sampler=sampler_engression, direction='minimize')
study_engression.optimize(engressor_NN, n_trials=N_TRIALS)

[I 2024-01-09 17:19:35,587] A new study created in memory with name: no-name-e8dcd721-fd53-4290-b8a6-fcad6c9bc482


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002880 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3766
[LightGBM] [Info] Number of data points in the train set: 5242, number of used features: 21
[LightGBM] [Info] Start training from score 91.111980


[I 2024-01-09 17:19:36,267] Trial 0 finished with value: 22.871447920679902 and parameters: {'learning_rate': 0.12071779104534666, 'n_estimators': 108, 'reg_lambda': 0.005044685709888605, 'max_depth': 23, 'min_child_samples': 55}. Best is trial 0 with value: 22.871447920679902.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000568 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3766
[LightGBM] [Info] Number of data points in the train set: 5242, number of used features: 21
[LightGBM] [Info] Start training from score 91.111980


[I 2024-01-09 17:19:36,526] Trial 1 finished with value: 23.787172223598073 and parameters: {'learning_rate': 0.004043145805966843, 'n_estimators': 179, 'reg_lambda': 0.0699481785242808, 'max_depth': 6, 'min_child_samples': 18}. Best is trial 0 with value: 22.871447920679902.


...

[I 2024-01-09 17:23:47,825] Trial 2 finished with value: 58.36392593383789 and parameters: {'learning_rate': 0.07075637776590661, 'num_epoches': 959, 'num_layer': 2, 'hidden_dim': 76, 'noise_dim': 91}. Best is trial 1 with value: 11.905889511108398.


Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1000.
[Epoch 1 (0%), batch 6] energy-loss: 0.7008,  E(|Y-Yhat|): 1.2413,  E(|Yhat-Yhat'|): 1.0810
[Epoch 100 (13%), batch 6] energy-loss: 0.2129,  E(|Y-Yhat|): 0.3976,  E(|Yhat-Yhat'|): 0.3693
[Epoch 200 (27%), batch 6] energy-loss: 0.2157,  E(|Y-Yhat|): 0.4177,  E(|Yhat-Yhat'|): 0.4039
[Epoch 300 (40%), batch 6] energy-loss: 0.2099,  E(|Y-Yhat|): 0.4005,  E(|Yhat-Yhat'|): 0.3813
[Epoch 400 (53%), batch 6] energy-loss: 0.2069,  E(|Y-Yhat|): 0.3826,  E(|Yhat-Yhat'|): 0.3513
[Epoch 500 (67%), batch 6] energy-loss: 0.1762,  E(|Y-Yhat|): 0.3536,  E(|Yhat-Yhat'|): 0.3549
[Epoch 600 (80%), batch 6] energy-loss: 0.2083,  E(|Y-Yhat|): 0.3914,  E(|Yhat-Yhat'|): 0.3662
[Epoch 700 (93%), batch 6] energy-loss: 0.

[I 2024-01-09 17:25:45,068] Trial 3 finished with value: 72.61552429199219 and parameters: {'learning_rate': 0.044997613517186334, 'num_epoches': 750, 'num_layer': 3, 'hidden_dim': 96, 'noise_dim': 86}. Best is trial 1 with value: 11.905889511108398.


Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1000.
[Epoch 1 (0%), batch 6] energy-loss: 0.3242,  E(|Y-Yhat|): 0.7643,  E(|Yhat-Yhat'|): 0.8801
[Epoch 100 (43%), batch 6] energy-loss: 0.1910,  E(|Y-Yhat|): 0.4149,  E(|Yhat-Yhat'|): 0.4479
[Epoch 200 (87%), batch 6] energy-loss: 0.1900,  E(|Y-Yhat|): 0.3945,  E(|Yhat-Yhat'|): 0.4091

Training loss on the original (non-standardized) scale:
	Energy-loss: 1.0098,  E(|Y-Yhat|): 2.1299,  E(|Yhat-Yhat'|): 2.2402

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a mismatch 


[I 2024-01-09 17:26:20,537] Trial 4 finished with value: 30.41522979736328 and parameters: {'learning_rate': 0.029128020748551788, 'num_epoches': 228, 'num_layer': 3, 'hidden_dim': 84, 'noise_dim': 72}. Best is trial 1 with value: 11.905889511108398.


In [ ]:
boosted_model.fit(X_train, y_train)
y_test_hat_boosted=boosted_model.predict(X_test)
RMSE_boosted=np.sqrt(np.mean((y_test-y_test_hat_boosted)**2))

rf_model.fit(X_train, y_train)
y_test_hat_rf=rf_model.predict(X_test)
RMSE_rf=np.sqrt(np.mean((y_test-y_test_hat_rf)**2))

lin_reg=LinearRegression()
lin_reg.fit(X_train, y_train)
y_test_hat_linreg=lin_reg.predict(X_test)
RMSE_linreg=np.sqrt(np.mean((y_test-y_test_hat_linreg)**2))

params=study_engression.best_params
engressor_model=engression(torch.Tensor(np.array(X_train)), torch.Tensor(np.array(y_train).reshape(-1,1)), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=1000)
y_test_hat_engression=engressor_model.predict(torch.Tensor(np.array(X_test)), target="mean")
RMSE_engression=np.sqrt((((torch.Tensor(np.array(y_test).reshape(-1,1)))-y_test_hat_engression)**2).mean(axis=0))

print("RMSE linear regression: ",RMSE_linreg)
print("RMSE boosted trees", RMSE_boosted)
print("RMSE random forest", RMSE_rf)
print("RMSE engression", RMSE_engression)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002846 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4351
[LightGBM] [Info] Number of data points in the train set: 6553, number of used features: 21
[LightGBM] [Info] Start training from score 88.320464
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

#### Extra stuff

#### Old way to define MLP

N_TRIALS=5

def MLP_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    n_layers = trial.suggest_int("n_layers", 1, 5)
    layers = []

    in_features = X_train_.shape[1]
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_uniform("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, 1))
    MLP_model=nn.Sequential(*layers)
    n_epochs=trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05)
    optimizer=torch.optim.Adam(MLP_model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()
    loss_Adam=[]

    for i in range(n_epochs):
        # making a pridiction in forward pass
        y_train_hat = MLP_model(torch.Tensor(X_train_.values)).reshape(-1,)
        # calculating the loss between original and predicted data points
        loss = criterion(y_train_hat, torch.Tensor(y_train_.values))
        # store loss into list
        loss_Adam.append(loss.item())
        # zeroing gradients after each iteration
        optimizer.zero_grad()
        # backward pass for computing the gradients of the loss w.r.t to learnable parameters
        loss.backward()
        # updating the parameters after each iteration
        optimizer.step()

    y_val_hat_MLP=(MLP_model(torch.Tensor(X_val.values)).reshape(-1,)).detach().numpy()
    RMSE_MLP=np.sqrt(np.mean((y_val-y_val_hat_MLP)**2))

    return RMSE_MLP

sampler_MLP = optuna.samplers.TPESampler(seed=10)
study_MLP = optuna.create_study(sampler=sampler_MLP, direction='minimize')
study_MLP.optimize(MLP_opt, n_trials=N_TRIALS)

#gp_model=GaussianProcessRegressor(kernel=Matern(length_scale=study_gp.best_params['lenghtscale'], nu=1.5))

In [ ]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

layers = []

in_features = X_train.shape[1]
for i in range(study_MLP.best_params['n_layers']):
    out_features = study_MLP.best_params["n_units_l{}".format(i)]
    layers.append(nn.Linear(in_features, out_features))
    layers.append(nn.ReLU())
    p = study_MLP.best_params["dropout_l{}".format(i)]
    layers.append(nn.Dropout(p))

    in_features = out_features

layers.append(nn.Linear(in_features, 1))
MLP_model=nn.Sequential(*layers)
n_epochs=study_MLP.best_params['n_epochs']
learning_rate=study_MLP.best_params['learning_rate']
optimizer=torch.optim.Adam(MLP_model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()
loss_Adam=[]

for i in range(n_epochs):
    # making a pridiction in forward pass
    y_train_hat = MLP_model(torch.Tensor(X_train.values)).reshape(-1,)
    # calculating the loss between original and predicted data points
    loss = criterion(y_train_hat, torch.Tensor(y_train.values))
    # store loss into list
    loss_Adam.append(loss.item())
    # zeroing gradients after each iteration
    optimizer.zero_grad()
    # backward pass for computing the gradients of the loss w.r.t to learnable parameters
    loss.backward()
    # updating the parameters after each iteration
    optimizer.step()

y_test_hat_MLP=(MLP_model(torch.Tensor(X_test.values)).reshape(-1,)).detach().numpy()
RMSE_MLP=np.sqrt(np.mean((y_test-y_test_hat_MLP)**2))
print("RMSE MLP: ", RMSE_MLP)